In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important;} </style>"))

In [2]:
import os
import datetime
import sys
import glob
import random
import pickle

import numpy         as np
import pandas        as pd

from scipy.fftpack   import fft
from pydub           import AudioSegment
from IPython.display import Audio

from pydub.silence   import split_on_silence
from pydub.silence   import detect_leading_silence

from tqdm            import tqdm

In [3]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

<h1 style="color:red"> Notes: </h1>
<li style="color:red"> 1.3 seconds sleep between files of same speaker </li>
<li style="color:red"> for simplicity - work with clients which has at least 3 tactic files </li>
<li style="color:red"> 10 seconds sleep between speakers </li>

<h1 style="background-color:LightGreen;"> <center> <a id='start_cell'></a> Table Of Contents </center></h1>

[Create Datasets](#create_dataset) </br>
[Players](#player) </br>
[Parse Log files](#parseLog) </br>
[Reciever](#Reciever) </br>
[Parse Records](#ParseRecords) </br>
[Plot dBFS](#Plot_dBFS) </br>

<h1 style="background-color:#3cA8EF;"> <center> <a id='create_dataset'></a> Create Clean Dataset Files </center> </h1>

In [4]:
if sys.platform != "win32":
    # linux
    ARABIC_PATH  = "/home/amitli/Datasets/CV_14/cv-corpus-14.0-2023-06-23-ar/cv-corpus-14.0-2023-06-23/ar"
    TURKISH_PATH = "/home/amitli/Datasets/CV_14/cv-corpus-14.0-2023-06-23-tr/cv-corpus-14.0-2023-06-23/tr"
    RUSSION_PATH = ""
else:
    # windows
    ARABIC_PATH  = r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ar"
    TURKISH_PATH = r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\tr"
    RUSSIAN_PATH = r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ru"

In [ ]:
def create_dataset(df_input, dsType, minNumOfSamples=0):
    arr_client       = []
    arr_file         = []
    arr_type         = []
    arr_lang         = []
    arr_numOfSamples = []
    arr_noiseType    = []

    arr_uniuque_clients = df_input['client_id'].unique()
    for i in range(len(arr_uniuque_clients)):
        
        client_id    = arr_uniuque_clients[i]   
        numOfSamples = len(df_input[df_input.client_id == client_id])
        
        if (numOfSamples >= minNumOfSamples) and (numOfSamples < 1000):

            for j in range(numOfSamples):
                file   = df_input[df_input.client_id == client_id]['path'].values[j]
                lang   = df_input[df_input.client_id == client_id]['locale'].values[j]
                dsType = dsType
                if j <= 0.33 * numOfSamples:
                    noiseType = "clean"
                elif j <= 0.66 * numOfSamples:
                    noiseType = "tactic"
                else:
                    noiseType = "background"

                arr_client.append(client_id)
                arr_file.append(file)
                arr_lang.append(lang)
                arr_type.append(dsType)
                arr_numOfSamples.append(numOfSamples)
                arr_noiseType.append(noiseType)                
                
                
    df = pd.DataFrame({"client_id":         arr_client,
                       "file":              arr_file,
                       "lang":              arr_lang,
                       "type":              arr_type,
                       "totalNumOfSamples": arr_numOfSamples,
                       "noiseType":         arr_noiseType})
    
    return df
        

<h1> Create Turkish: </h1>

In [ ]:
dataset_path = TURKISH_PATH

In [ ]:
train_df = pd.read_csv(dataset_path + "/train.tsv", delimiter="\t")
test_df  = pd.read_csv(dataset_path + "/test.tsv",  delimiter="\t")

In [ ]:
test_df['segment'].columnsmns

In [ ]:
df_train = create_dataset(train_df, "train", minNumOfSamples=20)
df_test  = create_dataset(test_df, "test",   minNumOfSamples=0)

In [ ]:
df_turkish = pd.concat([df_train, df_test])
df_turkish.head()
df_turkish.to_csv("./turkish.csv")

In [ ]:
df_turkish.shape

<h1> Create Arabic: </h1>

In [ ]:
dataset_path = ARABIC_PATH

In [ ]:
train_df = pd.read_csv(dataset_path + "/train.tsv", delimiter="\t")
test_df  = pd.read_csv(dataset_path + "/test.tsv",  delimiter="\t")

In [ ]:
df_train = create_dataset(train_df, "train", minNumOfSamples=20)
df_test  = create_dataset(test_df, "test",   minNumOfSamples=0)

In [ ]:
df_arabic = pd.concat([df_train, df_test])
df_arabic.head()
df_arabic.to_csv("./arabic.csv")

In [ ]:
df_arabic.shape

<h1> Create Russian: </h1>

In [ ]:
dataset_path = RUSSIAN_PATH

In [ ]:
train_df = pd.read_csv(dataset_path + "/train.tsv", delimiter="\t")
test_df  = pd.read_csv(dataset_path + "/test.tsv",  delimiter="\t")

In [ ]:
df_train = create_dataset(train_df, "train", minNumOfSamples=20)
df_test  = create_dataset(test_df, "test",   minNumOfSamples=0)

In [ ]:
df_russian = pd.concat([df_train, df_test])
df_russian.head()
df_russian.to_csv("./russian.csv")

In [ ]:
df_russian.shape

<h1 style="background-color:#3cA8EF;"> <center> Load </center> </h1>

In [5]:
df_arabic    = pd.read_csv("./arabic.csv")
df_turkish   = pd.read_csv("./turkish.csv")
df_russian   = pd.read_csv("./russian.csv")

In [ ]:
df_arabic[df_arabic.noiseType == "clean"].head()

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Player </center> </h1>

In [6]:
from pygame import mixer
import time
import IPython
import os

pygame 2.4.0 (SDL 2.26.4, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
num_to_freq_file = {}

num_to_freq_file[0]  = 'encoded_tones/audiocheck.net_sin_500Hz_-3dBFS_1.5s.wav'
num_to_freq_file[1]  = 'encoded_tones/audiocheck.net_sin_1000Hz_-10dBFS_1s.wav'
num_to_freq_file[2]  = 'encoded_tones/audiocheck.net_sin_1100Hz_-10dBFS_1s.wav'
num_to_freq_file[3]  = 'encoded_tones/audiocheck.net_sin_1200Hz_-10dBFS_1s.wav'
num_to_freq_file[4]  = 'encoded_tones/audiocheck.net_sin_1300Hz_-10dBFS_1s.wav'
num_to_freq_file[5]  = 'encoded_tones/audiocheck.net_sin_1400Hz_-10dBFS_1s.wav'
num_to_freq_file[6]  = 'encoded_tones/audiocheck.net_sin_1500Hz_-10dBFS_1s.wav'
num_to_freq_file[7]  = 'encoded_tones/audiocheck.net_sin_1600Hz_-10dBFS_1s.wav'
num_to_freq_file[8]  = 'encoded_tones/audiocheck.net_sin_1700Hz_-10dBFS_1s.wav'
num_to_freq_file[9]  = 'encoded_tones/audiocheck.net_sin_1800Hz_-10dBFS_1s.wav'
num_to_freq_file[10] = 'encoded_tones/audiocheck.net_sin_1900Hz_-10dBFS_1s.wav'

for f in num_to_freq_file.values():
    if not os.path.exists(f):
        print(f"Missing encoding file: {f}")

In [8]:
def play_file(filename, fileLengthInSeconds):
    mixer.music.load(filename)
    mixer.music.play()
    time.sleep(fileLengthInSeconds)
    time.sleep(0.1)
    while mixer.music.get_busy():  
        print("\n---> busy - wait -----\n")
        time.sleep(1)
    time.sleep(0.2)

In [9]:
def play_language(df, dataset_path, df_type, start_from_client=0):
        
    current_date = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S')    
    train_files  = list(set(list(df[df['type'] == df_type]['client_id'].values)))
    all_clients  = train_files

    mixer.init()     
    start_time = time.time()
    with open(f"log_{current_date}.txt", "w") as f:
        for c_i in range(len(all_clients)):  
            
            client_id    = all_clients[c_i]            
            lang         = df[df.client_id == client_id]['lang'].values[0]
            client_files = df[df.client_id == client_id]        
            client_files = client_files[client_files['noiseType'] == 'tactic']['file'].values
                                    
            ignore = False
            if len(client_files) < 3:
                ignore = True

            str_log   = f"\n\nClient_Id: {client_id} [{c_i+1}/{len(all_clients)}] Tactic files: {len(client_files)}, Ignore: {ignore}"        
            print(str_log)
            f.write(str_log)

            if ignore is True:
                continue
                
                
            if c_i+1 < start_from_client:
                str_log   = f"\n\nClient_Id: {client_id} Ignored start_from_client: {start_from_client}"        
                print(str_log)
                f.write(str_log)
                continue

            for file_i in range(len(client_files)):      
                
                file         = client_files[file_i]
                fullFilePath = f"{dataset_path}/clips/{file}"
                fileLength   = AudioSegment.from_mp3(fullFilePath).duration_seconds
                           
                end_time = time.time()
                str_log = f"\n[{file_i+1}/{len(client_files)}] Start Play file: {file}, Elapse: {round(end_time-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                
                play_file(fullFilePath, fileLength)
                end_start = time.time()

                str_log = f"\n[{file_i+1}/{len(client_files)}] ---> Played file: {file}, length: {fileLength} seconds, lang: {lang}, Elapse: {round(end_start-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                time.sleep(1)  
                  
            time.sleep(10)  
            
        end_time = time.time()        
        str_log = f"\nClose File, Elapse: {round(end_time-start_time, 3)}"
        print(str_log)
        f.write(str_log)

        f.close()

In [10]:
def beeps_play_file(filename, fileLengthInSeconds):
    mixer.music.load(filename)
    mixer.music.play()
    time.sleep(fileLengthInSeconds)
    time.sleep(0.1)
    while mixer.music.get_busy():  
        print("\n---> busy - wait -----\n")
        time.sleep(1)    

def beeps_play_language(df, dataset_path, df_type, start_from_client=0):
        
    current_date = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S')    
    train_files  = list(set(list(df[df['type'] == df_type]['client_id'].values)))
    all_clients  = train_files

    mixer.init()     
    start_time = time.time()
    with open(f"log_{current_date}.txt", "w") as f:
        for c_i in range(len(all_clients)):  
            
            client_id    = all_clients[c_i]            
            lang         = df[df.client_id == client_id]['lang'].values[0]
            client_files = df[df.client_id == client_id]        
            client_files = client_files[client_files['noiseType'] == 'tactic']['file'].values
                                    
            ignore = False
            if len(client_files) < 3:
                ignore = True

            str_log   = f"\n\nClient_Id: {client_id} [{c_i+1}/{len(all_clients)}] Tactic files: {len(client_files)}, Ignore: {ignore}"        
            print(str_log)
            f.write(str_log)

            if ignore is True:
                continue
                
                
            if c_i+1 < start_from_client:
                str_log   = f"\n\nClient_Id: {client_id} Ignored start_from_client: {start_from_client}"        
                print(str_log)
                f.write(str_log)
                continue

            for file_i in range(len(client_files)):      
                
                file         = client_files[file_i]
                fullFilePath = f"{dataset_path}/clips/{file}"
                fileLength   = AudioSegment.from_mp3(fullFilePath).duration_seconds
                           
                end_time = time.time()
                str_log = f"\n[{file_i+1}/{len(client_files)}] Start Play file: {file}, Elapse: {round(end_time-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                
                beeps_play_file('encoded_tones/audiocheck.net_sin_1000Hz_-10dBFS_1s.wav', 1)
                beeps_play_file(fullFilePath, fileLength)
                end_start = time.time()

                str_log = f"\n[{file_i+1}/{len(client_files)}] ---> Played file: {file}, length: {fileLength} seconds, lang: {lang}, Elapse: {round(end_start-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                time.sleep(1)  
                  
            time.sleep(10)  
            
        end_time = time.time()        
        str_log = f"\nClose File, Elapse: {round(end_time-start_time, 3)}"
        print(str_log)
        f.write(str_log)

        f.close()

In [ ]:
# play_language(df_arabic, ARABIC_PATH, "train")
# time.sleep(14)  
#play_language(df_turkish, TURKISH_PATH, "train") 
#time.sleep(14)  
#play_language(df_arabic, ARABIC_PATH, "test")
# time.sleep(14)  
#play_language(df_turkish, TURKISH_PATH, "test") 

# play_language(df_russian, RUSSIAN_PATH, "train") 
# time.sleep(5)  
#play_language(df_russian, RUSSIAN_PATH, "test") 



<H1 style="color:green">Shiri:</H1>

In [15]:
beeps_play_language(df_arabic, ARABIC_PATH, "train") 
#beeps_play_language(df_russian, RUSSIAN_PATH, "train") 
#beeps_play_language(df_russian, RUSSIAN_PATH, "test") 



Client_Id: fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a233c413e9271695f8e33e209c8b52065c951b7b4e8eef446a928074b5e28b9961a6f4c6cb9849325c7 [1/12] Tactic files: 175, Ignore: False

[1/175] Start Play file: common_voice_ar_24082982.mp3, Elapse: 0.784

[1/175] ---> Played file: common_voice_ar_24082982.mp3, length: 6.12 seconds, lang: ar, Elapse: 8.107

[2/175] Start Play file: common_voice_ar_24082986.mp3, Elapse: 9.222

[2/175] ---> Played file: common_voice_ar_24082986.mp3, length: 5.688 seconds, lang: ar, Elapse: 16.113

[3/175] Start Play file: common_voice_ar_24082988.mp3, Elapse: 17.257

[3/175] ---> Played file: common_voice_ar_24082988.mp3, length: 3.24 seconds, lang: ar, Elapse: 21.7

[4/175] Start Play file: common_voice_ar_24082991.mp3, Elapse: 22.821

[4/175] ---> Played file: common_voice_ar_24082991.mp3, length: 5.616 seconds, lang: ar, Elapse: 29.638

[5/175] Start Play file: common_voice_ar_24082992.mp3, Elapse: 31.414

[5/175] ---> Played file: common_voice_ar_2408299


[46/175] Start Play file: common_voice_ar_24083104.mp3, Elapse: 327.807

[46/175] ---> Played file: common_voice_ar_24083104.mp3, length: 5.688 seconds, lang: ar, Elapse: 334.699

[47/175] Start Play file: common_voice_ar_24083105.mp3, Elapse: 335.889

[47/175] ---> Played file: common_voice_ar_24083105.mp3, length: 3.168 seconds, lang: ar, Elapse: 340.26

[48/175] Start Play file: common_voice_ar_24083106.mp3, Elapse: 341.408

[48/175] ---> Played file: common_voice_ar_24083106.mp3, length: 3.6 seconds, lang: ar, Elapse: 346.209

[49/175] Start Play file: common_voice_ar_24083348.mp3, Elapse: 347.377

[49/175] ---> Played file: common_voice_ar_24083348.mp3, length: 3.168 seconds, lang: ar, Elapse: 351.748

[50/175] Start Play file: common_voice_ar_24083349.mp3, Elapse: 352.906

[50/175] ---> Played file: common_voice_ar_24083349.mp3, length: 3.528 seconds, lang: ar, Elapse: 357.636

[51/175] Start Play file: common_voice_ar_24083350.mp3, Elapse: 359.405

[51/175] ---> Played file: co


[92/175] Start Play file: common_voice_ar_24083416.mp3, Elapse: 641.103

[92/175] ---> Played file: common_voice_ar_24083416.mp3, length: 6.408 seconds, lang: ar, Elapse: 648.714

[93/175] Start Play file: common_voice_ar_24083417.mp3, Elapse: 649.9

[93/175] ---> Played file: common_voice_ar_24083417.mp3, length: 4.248 seconds, lang: ar, Elapse: 655.351

[94/175] Start Play file: common_voice_ar_24083418.mp3, Elapse: 656.535

[94/175] ---> Played file: common_voice_ar_24083418.mp3, length: 6.516 seconds, lang: ar, Elapse: 664.255

[95/175] Start Play file: common_voice_ar_24083419.mp3, Elapse: 666.024

[95/175] ---> Played file: common_voice_ar_24083419.mp3, length: 5.148 seconds, lang: ar, Elapse: 672.377

[96/175] Start Play file: common_voice_ar_24083420.mp3, Elapse: 673.528

[96/175] ---> Played file: common_voice_ar_24083420.mp3, length: 3.276 seconds, lang: ar, Elapse: 678.006

[97/175] Start Play file: common_voice_ar_24083421.mp3, Elapse: 679.169

[97/175] ---> Played file: c


[137/175] ---> Played file: common_voice_ar_24083492.mp3, length: 3.708 seconds, lang: ar, Elapse: 977.065

[138/175] Start Play file: common_voice_ar_24083495.mp3, Elapse: 978.237

[138/175] ---> Played file: common_voice_ar_24083495.mp3, length: 3.348 seconds, lang: ar, Elapse: 982.788

[139/175] Start Play file: common_voice_ar_24083496.mp3, Elapse: 983.93

[139/175] ---> Played file: common_voice_ar_24083496.mp3, length: 6.48 seconds, lang: ar, Elapse: 991.615

[140/175] Start Play file: common_voice_ar_24083497.mp3, Elapse: 992.789

[140/175] ---> Played file: common_voice_ar_24083497.mp3, length: 2.88 seconds, lang: ar, Elapse: 996.873

[141/175] Start Play file: common_voice_ar_24083498.mp3, Elapse: 998.044

[141/175] ---> Played file: common_voice_ar_24083498.mp3, length: 8.496 seconds, lang: ar, Elapse: 1007.744

[142/175] Start Play file: common_voice_ar_24083499.mp3, Elapse: 1009.525

[142/175] ---> Played file: common_voice_ar_24083499.mp3, length: 4.536 seconds, lang: ar,


[6/330] ---> Played file: common_voice_ar_24025958.mp3, length: 4.176 seconds, lang: ar, Elapse: 1286.332

[7/330] Start Play file: common_voice_ar_24025959.mp3, Elapse: 1287.511

[7/330] ---> Played file: common_voice_ar_24025959.mp3, length: 6.948 seconds, lang: ar, Elapse: 1295.663

[8/330] Start Play file: common_voice_ar_24025960.mp3, Elapse: 1296.819

[8/330] ---> Played file: common_voice_ar_24025960.mp3, length: 3.636 seconds, lang: ar, Elapse: 1301.658

[9/330] Start Play file: common_voice_ar_24025961.mp3, Elapse: 1302.815

[9/330] ---> Played file: common_voice_ar_24025961.mp3, length: 4.32 seconds, lang: ar, Elapse: 1308.337

[10/330] Start Play file: common_voice_ar_24025962.mp3, Elapse: 1309.483

[10/330] ---> Played file: common_voice_ar_24025962.mp3, length: 5.256 seconds, lang: ar, Elapse: 1315.943

[11/330] Start Play file: common_voice_ar_24025963.mp3, Elapse: 1317.684

[11/330] ---> Played file: common_voice_ar_24025963.mp3, length: 6.66 seconds, lang: ar, Elapse: 

...


[116/318] Start Play file: common_voice_ar_24069390.mp3, Elapse: 19279.273

[116/318] ---> Played file: common_voice_ar_24069390.mp3, length: 3.636 seconds, lang: ar, Elapse: 19284.118

[117/318] Start Play file: common_voice_ar_24069395.mp3, Elapse: 19285.271

[117/318] ---> Played file: common_voice_ar_24069395.mp3, length: 4.176 seconds, lang: ar, Elapse: 19290.651

[118/318] Start Play file: common_voice_ar_24069397.mp3, Elapse: 19291.816

[118/318] ---> Played file: common_voice_ar_24069397.mp3, length: 2.808 seconds, lang: ar, Elapse: 19295.827

[119/318] Start Play file: common_voice_ar_24069405.mp3, Elapse: 19297.591

[119/318] ---> Played file: common_voice_ar_24069405.mp3, length: 2.988 seconds, lang: ar, Elapse: 19301.782

[120/318] Start Play file: common_voice_ar_24069407.mp3, Elapse: 19302.926

[120/318] ---> Played file: common_voice_ar_24069407.mp3, length: 2.7 seconds, lang: ar, Elapse: 19306.829

[121/318] Start Play file: common_voice_ar_24069408.mp3, Elapse: 19307.


[160/318] ---> Played file: common_voice_ar_24069617.mp3, length: 3.528 seconds, lang: ar, Elapse: 19566.382

[161/318] Start Play file: common_voice_ar_24069620.mp3, Elapse: 19567.545

[161/318] ---> Played file: common_voice_ar_24069620.mp3, length: 6.516 seconds, lang: ar, Elapse: 19575.262

[162/318] Start Play file: common_voice_ar_24069621.mp3, Elapse: 19576.406

[162/318] ---> Played file: common_voice_ar_24069621.mp3, length: 4.608 seconds, lang: ar, Elapse: 19582.218

[163/318] Start Play file: common_voice_ar_24069622.mp3, Elapse: 19583.362

[163/318] ---> Played file: common_voice_ar_24069622.mp3, length: 6.156 seconds, lang: ar, Elapse: 19590.72

[164/318] Start Play file: common_voice_ar_24070147.mp3, Elapse: 19592.476

[164/318] ---> Played file: common_voice_ar_24070147.mp3, length: 5.868 seconds, lang: ar, Elapse: 19599.549

[165/318] Start Play file: common_voice_ar_24070148.mp3, Elapse: 19600.672

[165/318] ---> Played file: common_voice_ar_24070148.mp3, length: 2.73


[205/318] Start Play file: common_voice_ar_24070511.mp3, Elapse: 19853.754

[205/318] ---> Played file: common_voice_ar_24070511.mp3, length: 2.808 seconds, lang: ar, Elapse: 19857.766

[206/318] Start Play file: common_voice_ar_24070515.mp3, Elapse: 19858.881

[206/318] ---> Played file: common_voice_ar_24070515.mp3, length: 8.568 seconds, lang: ar, Elapse: 19868.652

[207/318] Start Play file: common_voice_ar_24070516.mp3, Elapse: 19870.426

[207/318] ---> Played file: common_voice_ar_24070516.mp3, length: 2.736 seconds, lang: ar, Elapse: 19874.366

[208/318] Start Play file: common_voice_ar_24070528.mp3, Elapse: 19875.527

[208/318] ---> Played file: common_voice_ar_24070528.mp3, length: 4.86 seconds, lang: ar, Elapse: 19881.588

[209/318] Start Play file: common_voice_ar_24070531.mp3, Elapse: 19882.708

[209/318] ---> Played file: common_voice_ar_24070531.mp3, length: 3.528 seconds, lang: ar, Elapse: 19887.438

[210/318] Start Play file: common_voice_ar_24070532.mp3, Elapse: 19888


[249/318] ---> Played file: common_voice_ar_24073188.mp3, length: 3.456 seconds, lang: ar, Elapse: 20136.219

[250/318] Start Play file: common_voice_ar_24073189.mp3, Elapse: 20137.4

[250/318] ---> Played file: common_voice_ar_24073189.mp3, length: 2.736 seconds, lang: ar, Elapse: 20141.339

[251/318] Start Play file: common_voice_ar_24073192.mp3, Elapse: 20143.1

[251/318] ---> Played file: common_voice_ar_24073192.mp3, length: 2.556 seconds, lang: ar, Elapse: 20146.86

[252/318] Start Play file: common_voice_ar_24073213.mp3, Elapse: 20147.979

[252/318] ---> Played file: common_voice_ar_24073213.mp3, length: 2.988 seconds, lang: ar, Elapse: 20152.169

[253/318] Start Play file: common_voice_ar_24073216.mp3, Elapse: 20153.336

[253/318] ---> Played file: common_voice_ar_24073216.mp3, length: 3.96 seconds, lang: ar, Elapse: 20158.496

[254/318] Start Play file: common_voice_ar_24073218.mp3, Elapse: 20159.64

[254/318] ---> Played file: common_voice_ar_24073218.mp3, length: 2.556 seco


[294/318] Start Play file: common_voice_ar_24083719.mp3, Elapse: 20404.723

[294/318] ---> Played file: common_voice_ar_24083719.mp3, length: 3.456 seconds, lang: ar, Elapse: 20409.383

[295/318] Start Play file: common_voice_ar_24083721.mp3, Elapse: 20411.127

[295/318] ---> Played file: common_voice_ar_24083721.mp3, length: 2.736 seconds, lang: ar, Elapse: 20415.065

[296/318] Start Play file: common_voice_ar_24083722.mp3, Elapse: 20416.22

[296/318] ---> Played file: common_voice_ar_24083722.mp3, length: 3.708 seconds, lang: ar, Elapse: 20421.131

[297/318] Start Play file: common_voice_ar_24083723.mp3, Elapse: 20422.263

[297/318] ---> Played file: common_voice_ar_24083723.mp3, length: 3.276 seconds, lang: ar, Elapse: 20426.74

[298/318] Start Play file: common_voice_ar_24083724.mp3, Elapse: 20427.891

[298/318] ---> Played file: common_voice_ar_24083724.mp3, length: 2.7 seconds, lang: ar, Elapse: 20431.794

[299/318] Start Play file: common_voice_ar_24083725.mp3, Elapse: 20432.94

<h1 style="background-color:#3cA8EF;"> <center><a id='parseLog'></a> Parse player log </center> </h1>

In [ ]:
def parse_log(log_file):
    
    arr_clients            = []   # arr of client_id [may be duplicate - depends on num of files per client]
    arr_cv                 = []   # cv file
    arr_time               = []   # cv file length
    arr_lang               = []   # language
    arr_num_cv_per_speaker = []   # for each speaker - number of speechs
    arr_start_speech_time  = []   # for each speech - start time
    arr_end_speech_time    = []

    with open(log_file) as f:
        lines = f.readlines()

        counter     = 0
        last_client = None
        start_time  = None        
        
        for line in lines:

            if line.find("Client_Id") != -1:
                
                client      = line[line.find(": ")+2 : line.find(" [")]
                last_client = client
                if counter > 0:
                    arr_num_cv_per_speaker.append(counter) 
                    counter = 0
                    
                 
            elif line.find("Start Play file:") != -1:                     
                start_time = float(line[line.find("Elapse:")+8:])                

                    
            elif line.find("---> Played file") != -1:                
                cv_file = line[line.find(": c")+2 : line.find(", ")]
                length  = float(line[line.find("length:")+8 : line.find(" seconds")])
                lang    = line[line.find("lang:")+6 : line.find("lang:")+8]         
                end_t   = float(line[line.find("Elapse: ")+8:])                
                counter = counter + 1
                
                arr_clients.append(last_client)
                arr_cv.append(cv_file)
                arr_time.append(length)
                arr_lang.append(lang)     
                arr_end_speech_time.append(end_t)
                if start_time is not None:
                    arr_start_speech_time.append(start_time)
                    start_time = None
                else:
                    print(f"Error, start_time is None")
           
            
    arr_num_cv_per_speaker.append(counter)         
    print(f"Total number of speakers: {len(set(arr_clients))}, files: {len(set(arr_cv))}, langs: {len(set(arr_lang))}")
    print(f"len(arr_start_speech_time) = {len(arr_start_speech_time)}")
    return arr_clients, arr_cv, arr_time, arr_lang, arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time

In [ ]:
NIGHT_RUN_1 = r"/home/amitli/Debug/25_26_jul/Logs/log_25_07_2023_15_18_39.txt"
arr_clients, arr_cv, arr_time, arr_lang, arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time = parse_log(NIGHT_RUN_1)

<h1 style="background-color:#3cA8EF;"> <center> <a id='Reciever'></a> Reciever </center> </h1>

In [ ]:
from dataclasses        import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor
from pydub              import AudioSegment,silence

import datetime
import pyaudio
import wave

In [ ]:

@dataclass
class StreamParams:
    format: int            = pyaudio.paInt16
    channels: int          = 1
    rate: int              = 8000
    frames_per_buffer: int = 1024
    input: bool            = True
    output: bool           = False

    def to_dict(self) -> dict:
        return asdict(self)

class Recorder:
    """Recorder uses the blocking I/O facility from pyaudio to record sound
    from mic.
    Attributes:
        - stream_params: StreamParams object with values for pyaudio Stream
            object
    """
    def __init__(self, stream_params: StreamParams) -> None:
        self.stream_params = stream_params
        self._pyaudio      = None
        self._stream       = None
        self._wav_file     = None
        self._counter      = 0

    def record(self, duration: int, save_path: str, num_files_to_create: int) -> None:
        """Record sound from mic for a given amount of seconds.
        :param duration: Number of seconds we want to record for
        :param save_path: Where to store recording
        """
        print("Start recording...")
        self.save_path = save_path
        self._create_recording_resources()
        self._write_wav_file_reading_from_stream(save_path, duration, num_files_to_create)
        self._close_recording_resources()
        print("Stop recording")

    def create_current_wav_file(self):
        
        self._counter = self._counter + 1 
        the_time  = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S%z")
        ctr_str   = f'{self._counter:04}'
        save_path = self.save_path.replace(".wav",f"_C_{ctr_str}_D_{the_time}.wav")        
        self._create_wav_file(save_path)
    
    def _create_recording_resources(self) -> None:
        self._pyaudio = pyaudio.PyAudio()
        self._stream = self._pyaudio.open(**self.stream_params.to_dict())
        self.create_current_wav_file()

    def _create_wav_file(self, save_path: str):
        print(f"creating new wav: {save_path}")
        self._wav_file = wave.open(save_path, "wb")
        self._wav_file.setnchannels(self.stream_params.channels)
        self._wav_file.setsampwidth(self._pyaudio.get_sample_size(self.stream_params.format))
        self._wav_file.setframerate(self.stream_params.rate)


    def close_current_wav_file(self, wav_file) -> None:
        wav_file.close()

    def _write_wav_file_reading_from_stream(self, save_path: str, duration: int, num_files_to_create: int) -> None:
        with ThreadPoolExecutor(max_workers = 5) as executor:
            for i in range(num_files_to_create):
                for _ in range(int(self.stream_params.rate * duration / self.stream_params.frames_per_buffer)):
                    audio_data = self._stream.read(self.stream_params.frames_per_buffer)
                    self._wav_file.writeframes(audio_data)
                    
                executor.submit(self.close_current_wav_file, self._wav_file)
                if i < num_files_to_create-1:
                    self.create_current_wav_file()

    def _close_recording_resources(self) -> None:
        self._stream.close()
        self._pyaudio.terminate()


In [ ]:
stream_params = StreamParams()
stream_params.rate = 32000
recorder = Recorder(stream_params)
recorder.record(60, "/home/amitli/Datasets/speakathon/audio.wav", num_files_to_create=60*18)

<h1 style="background-color:#3cA8EF;"> <center><a id='ParseRecords'></a>  Parse recordings </center> </h1>

In [ ]:
def get_sorted_files(files_path):
    all_files = glob.glob(files_path)
    all_files.sort()    
    return all_files    

In [ ]:
NIGHT_RUN_INPUT_FOLDER = r"/home/amitli/Debug/25_26_jul/Night_25_26/*"

In [ ]:
all_files = get_sorted_files(NIGHT_RUN_INPUT_FOLDER)
one_file  = AudioSegment.from_wav(all_files[0]) 
for i in range(1, len(all_files)):
    one_file = one_file + AudioSegment.from_wav(all_files[i])   

In [ ]:
NIGHT_RUN_1_ONE_FILE = r"/home/amitli/Debug/25_26_jul/One_file/One_file.wav"

In [ ]:
one_file.export(NIGHT_RUN_1_ONE_FILE, format="wav")

In [ ]:
arr_start_speech_time[:5]

In [ ]:
#AudioSegment.from_wav(r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav")[9.9*1000:11*1000]

In [ ]:
full_wav              = AudioSegment.from_wav(NIGHT_RUN_1_ONE_FILE)
mp3_len               = arr_time
num_of_speakers       = len(set(arr_clients))
num_of_mp3_per_speker = arr_num_cv_per_speaker


start_rec_time_ms = int(9.9*1000) - arr_start_speech_time[0] * 1000
arr_results       = []
counter           = 0
end               = 0

for i_speaker in range(num_of_speakers):
   
    for i_mp3 in range(num_of_mp3_per_speker[i_speaker]):
                
        # --- speech
        start = start_rec_time_ms + arr_start_speech_time[counter]*1000
        #end   = arr_end_speech_time[counter]*1000 + start_rec_time_ms
        end   = start + mp3_len[counter]*1000   
        
        
        if end > len(full_wav):
            break
                   
        arr_results.append(full_wav[start-300 : end])                   
        counter = counter + 1
           

print(f"Finished at = {end} / {len(full_wav)}, Found: {len(arr_results)}")

In [ ]:
test = 2354
arr_results[test]

In [ ]:
print(f"{arr_cv[test]}, {arr_time[test]}, recorded: {arr_results[test].duration_seconds}")
AudioSegment.from_mp3(f"{ARABIC_PATH}/clips/{arr_cv[test]}")

In [ ]:
last_speaker  = 0
export_path   = "/home/amitli/Debug/25_26_jul/Outputs_wav"
full_path     = ""
for i in tqdm(range(2993)):
    
    speaker     = arr_clients[i]
    if speaker != last_speaker:
        expoer_folder = speaker
        full_path     = f"{export_path}/{speaker}"
        if not os.path.exists(full_path):  
            os.makedirs(full_path)        
        
    last_speaker = speaker
    tactic_file  = arr_results[i]
    cv_name      = arr_cv[i]
    lang         = arr_lang[i]
    
    file_name    = f"tactic_{lang}_{cv_name}"
    #arr_results[i].export(f"{full_path}/{file_name}", format="mp3")
    arr_results[i].export(f"{full_path}/{file_name[:-4]}.wav", format="wav")    

<h1 style="background-color:pink;"> <center> Try with VAD </center> </h1>

In [ ]:
import torch
import librosa

vad_path   = 'snakers4/silero-vad'
vad_source = "github"

vad_model, vad_utils   = torch.hub.load(repo_or_dir=vad_path, model='silero_vad', source=vad_source, force_reload=False, onnx=False)
(vad_get_speech_timestamps, vad_save_audio, vad_read_audio, VADIterator, vad_collect_chunks) = vad_utils

In [ ]:
y, sr = librosa.load(NIGHT_RUN_1_ONE_FILE, sr=16000)

In [ ]:
speech_timestamps = vad_get_speech_timestamps(y,vad_model,sampling_rate=16000, min_silence_duration_ms=900, threshold=0.4)
print(len(speech_timestamps))

In [ ]:
i = 0
AudioSegment.from_wav(NIGHT_RUN_1_ONE_FILE)[speech_timestamps[i]['start'] : speech_timestamps[i]['end']]

<h1 style="background-color:#3cA8EF;"> <center> <a id='Plot_dBFS'></a>  Plot (dBFS) </center> </h1>

In [ ]:
import plotly.graph_objects as go
import numpy as np
import wave

def read_wav_file(file_path):
    with wave.open(file_path, "rb") as wav_file:
        num_frames   = wav_file.getnframes()
        sample_width = wav_file.getsampwidth()
        sample_rate  = wav_file.getframerate()
        audio_data   = np.frombuffer(wav_file.readframes(num_frames), dtype=np.int16) 
    return audio_data, sample_rate


def convert_to_dbfs(audio_data):
    max_value = np.max(np.abs(audio_data))
    dbfs = 20 * np.log10(audio_data.astype(np.float32) / max_value)
    return dbfs


def plot_dBFS(audio_data, sample_rate):
    duration = len(audio_data) / sample_rate
    time = np.linspace(0, duration, len(audio_data))
    dbfs = convert_to_dbfs(audio_data)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=dbfs, mode="lines"))
    fig.update_layout(
        title="dBFS Plot",
        xaxis_title="Time (s)",
        yaxis_title="dBFS",
    )
    return fig

    
file_path = r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav"
audio_data, sample_rate = read_wav_file(file_path)
fig                     = plot_dBFS(audio_data, sample_rate)
fig.show()    

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Verify </center> </h1>

In [ ]:
df_tactic = pd.read_csv(r"F:\tactic_final.csv")
print(f"total = {len(df_tactic)}")

In [ ]:
# import plotly.express as px
# data = df_tactic[df_tactic.common_voice_num_of_words == 3]['common_voice_file_len_sec'].values
# fig = px.histogram(data, nbins=30, title='Random Data Histogram')
# fig.update_xaxes(title_text='Value')
# fig.update_yaxes(title_text='Frequency')

# # Show the plot
# fig.show()

In [ ]:

def get_tactic(num_of_words = None, cv_file_len_sec = None, index=0):
    
    if cv_file_len_sec is not None:
        tmp_df    = df_tactic[df_tactic.common_voice_file_len_sec <= cv_file_len_sec]
    elif num_of_words is not None:
        tmp_df    = df_tactic[df_tactic.common_voice_num_of_words == num_of_words]
#     tmp_df = df_tactic[df_tactic.common_voice_num_of_words == 3]
#     tmp_df = tmp_df[tmp_df.common_voice_file_len_sec < 1.2]
    
    wav_file  = tmp_df['hackaton_file_name'].values[index]
    lang      = tmp_df['language'].values[index]
    cv_type   = tmp_df['common_voice_file_type'].values[index]
    speaker   = tmp_df['speaker'].values[index]    
    words     = tmp_df['common_voice_num_of_words'].values[index]    
    flen      = tmp_df['common_voice_file_len_sec'].values[index]    
    full_path = rf"F:\Tactic\{lang.capitalize()}\{lang}_{cv_type}\{speaker}\{wav_file}"
    print(f"num_of_words = {words}, length = {flen} ms")
    return full_path


full_path = get_tactic(num_of_words=None, cv_file_len_sec=1, index=10)
AudioSegment.from_wav(full_path)

In [ ]:
y, sr = librosa.load(full_path, sr=16000)
speech_timestamps = vad_get_speech_timestamps(y,vad_model,sampling_rate=16000, min_silence_duration_ms=900, threshold=0.4)
print(len(speech_timestamps))

<h1> collect all wavs which have no words </h1>

In [ ]:
vad_get_speech_timestamps?

In [ ]:
RUN_VAD  = True

if RUN_VAD is True:
    l_indexs = []
    l_tactic = []

    for i in tqdm(range(len(df_tactic))):

        wav_file          = df_tactic['hackaton_file_name'].values[i]
        speaker           = df_tactic['speaker'].values[i]
        cv_type           = df_tactic['common_voice_file_type'].values[i]
        lang              = df_tactic['language'].values[i]
        num_of_words      = df_tactic['common_voice_num_of_words'].values[i]
        len_secs          = df_tactic['common_voice_file_len_sec'].values[i]
        full_path         = rf"F:\Tactic\{lang.capitalize()}\{lang}_{cv_type}\{speaker}\{wav_file}"
        
        if num_of_words <= 2 and len_secs <= 2.1:
            y, sr             = librosa.load(full_path, sr=8000)
            speech_timestamps = vad_get_speech_timestamps(y,vad_model,sampling_rate=8000, threshold=0.4)

            if len(speech_timestamps) == 0:
                l_indexs.append(i)
                l_tactic.append(full_path)
        
    with open('wrong_tactic_inxs.pkl', 'wb') as f:
        pickle.dump(l_indexs, f)

    with open('wrong_tactic_fnames.pkl', 'wb') as f:
        pickle.dump(l_tactic, f)        
else:
    
    with open('wrong_tactic_inxs.pkl', 'rb') as f:
        l_indexs = pickle.load(f)
        
    with open('wrong_tactic_fnames.pkl', 'rb') as f:
        l_tactic = pickle.load(f)
        
print(f"Total: {len(l_tactic)}, Rate: {len(l_tactic) / len(df_tactic)}")

In [ ]:
l_tactic[0]

In [ ]:
AudioSegment.from_wav(l_tactic[15])

In [ ]:
df_f = pd.read_csv(r"f:\Hackathon_final_with_ds_spliter.csv")
df_f.head()

In [ ]:
def get_file(noise_type, index):
    tmp_df = df_f[df_f.hackathon_type == noise_type]
    cv     = tmp_df['common_voice_file_name'].values[index]
    hc     = tmp_df['final_file'].values[index]
    lang   = tmp_df['language'].values[index][:2]
    c_type = tmp_df['common_voice_file_type'].values[index]
    
    full_cv = rf"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\{lang}\clips\{cv}"
    full_hc = rf"F:\Hackathon_final\{hc}"
    
    return full_cv, full_hc
  
full_cv, full_hc = get_file(noise_type="tactic", index=1901)

In [ ]:
AudioSegment.from_mp3(full_cv)

In [ ]:
AudioSegment.from_wav(full_hc)

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Create new tactic names </center> </h1>

In [ ]:
df_f

In [ ]:
dict_fnames = {}

# --- get clean and background 
df_f        = pd.read_csv(r"f:\Hackathon_final_with_ds_spliter.csv")
tmp_df      = df_f[df_f.hackathon_type != 'tactic']
for i in range(len(tmp_df)):
    final_file = tmp_df['final_file'].values[i][:-4]    
    cv_file    = tmp_df['common_voice_file_name'].values[i]
    
    if final_file in dict_fnames.keys():
        print("error")    
    else:
        dict_fnames[final_file] = cv_file
        hack_wav                = AudioSegment.from_wav(rf"F:\Hackathon_final\{final_file}.wav")
        hack_wav.export(rf"F:\Hackathon_Dataset\{final_file}.wav", format="wav")                
        
        
#-- get background
tmp_df      = df_f[df_f.hackathon_type == 'tactic']
l = []
c = []
for i in range(len(tmp_df)):
    final_file  = tmp_df['final_file'].values[i][:-4]    
    cv_file     = tmp_df['common_voice_file_name'].values[i]
    
    lang        = tmp_df['language'].values[i]
    speaker     = tmp_df['speaker'].values[i]
    cv_type     = tmp_df['common_voice_file_type'].values[i]
    tactic_name = tmp_df['hackaton_file_name'].values[i]    
    tactic_path = rf"F:\Tactic\{lang.capitalize()}\{lang}_{cv_type}\{speaker}\{tactic_name}"
    
    if final_file in dict_fnames.keys():
        print("error")    
    else:
        dict_fnames[final_file] = cv_file
        tatic_wav               = AudioSegment.from_wav(tactic_path)
        tatic_wav.export(rf"F:\Hackathon_Dataset\{final_file}.wav", format="wav")        
        l.append(final_file)
        c.append(rf"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\{lang[:2]}\clips\{cv_file}")
        

        
print(len(dict_fnames))
print(len(df_f) == len(dict_fnames))

In [ ]:
AudioSegment.from_mp3(c[17011])

In [ ]:
l[0]

In [ ]:
AudioSegment.from_wav(rf"F:\Hackathon_Dataset\{l[17011]}.wav")

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Remove outliers </center> </h1>

In [ ]:
df_f        = pd.read_csv(r"f:\Hackathon_final_with_ds_spliter.csv")
len(df_f)

In [ ]:
l_removed_files = []
for i in range(len(l_tactic)):
    file = l_tactic[i][l_tactic[i].rfind('\\')+1:]
    l_removed_files.append(file)    
len(l_removed_files)

In [ ]:
l_remove_final  = []
for file in l_removed_files:    
    final_file = df_f[df_f.hackaton_file_name == file]['final_file'].values[0]
    l_remove_final.append(final_file)
len(l_remove_final)

In [ ]:
filtered_df = df_f[~df_f['hackaton_file_name'].isin(l_removed_files)]
len(filtered_df)

In [ ]:
filtered_df.to_csv(r"F:\Hackathon_Dataset\Hackathon_final_without_outliers.csv")

In [ ]:
for filename in l_remove_final:
    file_path = os.path.join(rf"F:\Hackathon_Dataset", filename)    
    try:
        os.remove(file_path)        
    except:
        print("error")

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Validate </center> </h1>

In [ ]:
df_Hackathon = pd.read_csv(r"f:\Hackathon_Dataset\Hackathon_final_without_outliers.csv")
i = 990
final_file = df_Hackathon[df_Hackathon.hackathon_type == 'tactic']['final_file'].values[i]
cv_file    = df_Hackathon[df_Hackathon.hackathon_type == 'tactic']['common_voice_file_name'].values[i]
lang       = df_Hackathon[df_Hackathon.hackathon_type == 'tactic']['language'].values[i] 
speaker    = df_Hackathon[df_Hackathon.hackathon_type == 'tactic']['speaker'].values[i] 
cv_type    = df_Hackathon[df_Hackathon.hackathon_type == 'tactic']['common_voice_file_type'].values[i]          

In [ ]:
AudioSegment.from_wav(rf"F:\Hackathon_Dataset\{final_file}")

In [ ]:
AudioSegment.from_mp3(rf"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\{lang[:2]}\clips\{cv_file}")

<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Balanced (remove some clean files) </center> </h1>

In [ ]:
df_Hackathon = pd.read_csv(r"f:\Hackathon_Dataset\Hackathon_final_without_outliers.csv")

In [ ]:
len(df_Hackathon)

In [ ]:
l_speakers = set(df_Hackathon['speaker'].values)
l_removed  = []

for speaker in l_speakers:
    count = len(df_Hackathon[df_Hackathon.speaker == speaker])
    if count > 400:
        df_tmp        = df_Hackathon[df_Hackathon.speaker == speaker]
        l_clean_files = df_tmp[df_tmp.hackathon_type == 'clean']['final_file'].values
        for file in l_clean_files:
            l_removed.append(file)
            
print(f"Total removed: {len(l_removed)}")

In [ ]:
df_f        = pd.read_csv(r"f:\Hackathon_Dataset\Hackathon_final_without_outliers.csv")
filtered_df = df_f[~df_f['final_file'].isin(l_removed)]
len(filtered_df)

In [ ]:
filtered_df.to_csv(r"F:\Hackathon_Dataset_Balanced\Balanced_Hackathon_final_without_outliers.csv")

In [ ]:
for filename in l_removed:
    file_path = os.path.join(rf"F:\\Hackathon_Dataset_Balanced", filename)        
    try:
        os.remove(file_path)                        
    except:
        print("error")

<h1 style="background-color:#3cA8EF;"> <center> Get SNR </center> </h1>

In [ ]:
df = pd.read_csv(r"F:\Hackathon_Dataset_Balanced\Balanced_Hackathon_final_without_outliers.csv")
df[df.hackathon_type == 'clean']['']

In [ ]:
import numpy as np
from scipy.io import wavfile

# Read the WAV file
sample_rate, audio_data = wavfile.read(r'F:\Hackathon_Dataset_Balanced\')

# Calculate signal power
signal_power = np.mean(audio_data ** 2)

# Calculate noise power (You may need to estimate this)
noise_power = np.mean(noise_data ** 2)  # Replace noise_data with your noise signal

# Calculate SNR in dB
snr_db = 10 * np.log10(signal_power / noise_power)

print(f"SNR: {snr_db:.2f} dB")